# 1 - Introduction to Catalog and Basic Python Client Interactions

## Before starting, follow installation here

### INSTALL REQUIREMENTS
1) Install Globus Catalog-client (https://github.com/globusonline/catalog-client)
git clone https://github.com/globusonline/catalog-client
cd catalog-client
python setup.py install --user

2) Install Globus Transfer API (https://github.com/globusonline/transfer-api-client-python)
git clone https://github.com/globusonline/transfer-api-client-python
cd transfer-api-client-python
python setup.py install --user

### Globus Online - Catalog Command Line Client
[https://github.com/globusonline/catalog-client]

Catalog User Interface: https://catalog-alpha.globuscs.info/
Contact: Ben Blaiszik (blaiszik@uchicago.edu)

### OBTAIN GLOBUS CREDENTIALS
https://www.globus.org/SignUp - **this is the credential you will use with Catalog**


# Catalog Data Model

* <b> Catalogs </b>
    * Have specified "vocabularies" or tag definitions.
        * e.g. beam_energy - float, decription - text
    * Catalogs contain many datasets
        
* <b> Datsets </b>
    * Datasets can have tags added and ACLs specified
    * Datasets contain many members
* <b> Members </b>
    * Members can have tags added
    * Generally point to a data file or directory on a Globus endpoint
    * COuld be a more general URI

<img src="img/catalog-model.png" width=70%>

In [12]:
# import scipy
# from scipy import *
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
%matplotlib inline

import os
from globusonline.catalog.client.catalog_wrapper import *
from globusonline.catalog.client.operators import Op
from globusonline.catalog.client.rest_client import RestClientError




# Authentication

In [13]:
# Store authentication data in a local file
token_file = os.getenv('HOME','')+"/.ssh/gotoken.txt"
wrap = CatalogWrapper(token_file=token_file)
client = wrap.catalogClient

# Create a Catalog and Save the ID

In [15]:
catalog_info = { 
                    "config": {
                        "name":"Ben Demo Catalog"
                    }
               }
_,response = client.create_catalog(catalog_info)
catalog_id = response['id']

In [20]:
response

{u'config': {u'content_read_users': [u'*'],
  u'content_write_users': [u'*'],
  u'name': u'Ben Demo Catalog',
  u'owner': u'u:blaiszik',
  u'read_users': [u'*'],
  u'write_users': []},
 u'id': 144}

# Create a Dataset within the New Catalog and Save the Dataset ID

In [21]:
dataset_info = {"name":"New Dataset"}
_,response = client.create_dataset(catalog_id, dataset_info)

In [22]:
response

{u'annotations_present': [u'created',
  u'id',
  u'modified',
  u'modified by',
  u'name',
  u'owner',
  u'readok',
  u'writeok'],
 u'created': u'2015-11-05 17:11:05.639515+00:00',
 u'id': 49,
 u'modified': u'2015-11-05 17:11:05.639515+00:00',
 u'modified by': u'u:blaiszik',
 u'name': u'New Dataset',
 u'owner': u'u:blaiszik',
 u'writeok': True}